In [22]:
from urllib import request as req
from bs4 import BeautifulSoup
import csv
import requests
import pandas as pd


ranks = ['S','A']
all_players = []  # 여기에 선수 이름들 저장

for rank in ranks:
    url = f"https://liquipedia.net/overwatch/{rank}-Tier_Tournaments"
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, 'html.parser')

    competition = []

    target_div = soup.select_one('div.gridTable.tournamentCard.Tierless.NoGameIcon')

    if target_div:
        a_tags = target_div.select('div.gridCell.Tournament.Header > a')
        for a in a_tags:
            href = a.get('href')
            if href:
                full_url = f"https://liquipedia.net{href}"
                competition.append(full_url)
    else:
        print("타겟 div를 찾을 수 없습니다.")

    for comp_url in competition:
        res = requests.get(comp_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        players = soup.select(
            '#mw-content-text div.teamcard.toggle-area.toggle-area-1 div.teamcard-inner table[data-toggle-area-content="1"] tbody tr td > a'
        )

        if players:
            for p in players:
                name = p.text.strip()
                all_players.append(name)  # 전역 리스트에 추가
        else:
            print("❌ 선수 또는 팀 정보를 찾을 수 없습니다.")

In [23]:
all_players = list(set(all_players))
# 모든 선수 이름 출력
print(all_players)

['tunder', 'speed75', 'Hesty', 'Zeb', 'pela', 'mazz', 'cookie084', 'Remaker', 'Aki', 'D4RT', 'FREGRE', 'pokotaros', 'Esteban', 'Eileen', 'hamster', 'Faith', 'revzi', 'Prophet', 'Kurumi', 'Taejong', 'WoochaN', 'D0NGHAK', 'UYOU', 'cuFFa', 'Seicoe', 'DML', 'Mealgaru', 'Youbi', 'ProYung', 'Tamper', 'StillKIWI', 'scuffed', 'TwolzZ', 'KSG', 'Khenail', 'yoham', 'R3K', 'crispy', 'Dip', 'Lengsa', 'Estreichh', 'gogo', 'Exrai', 'Juno', 'Simple', 'Danny', 'MAKA', 'Hry', 'Ado', 'Ewan', 'Xzahyo', 'Kellex', 'Architect', 'Insane', 'SKAI', 'Jasm1ne', 'Bilideng', 'slowdive', 'nuka', 'Genius91', 'Attack', 'decade', 'Twe12e', 'Viper', 'meruna', 'AOY', 'knife', 'GaaRa', 'JBJ', 'Ryujehong', 'Sauna', 'Despair', 'pOv', 'K4ne', 'Ken', 'Dumbbell', 'Love', 'Fone', 'Proud', 'FARMER', 'fya', 'AMANE', 'Aniyun', 'Langley', 'Leonopteryx', 'MER1T', 'Astro', 'KIVIS', 'lumi', 'Proxy', 'dra', 'seeker', 'Lukemino', 'winter', 'elva', 's9mm', 'WGYM', 'JayCo', 'PaLee', 'ZEST', 'Maybe', 'ARLLY', 'Cjay', 'Junbin', 'D0D0', 'KL1

In [24]:
fields = [
    "Player", "Romanized Name", "Nationality", "Born", "Status",
    "Role", "Team", "Signature Heroes", "Game Appearances", "Approx. Total Winnings"
]

def get_player_info(player_name):
    url = f"https://liquipedia.net/overwatch/{player_name}"
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, 'html.parser')

    info = {field: "" for field in fields}
    info["Player"] = player_name

    # Romanized Name 보완
    name_divs = soup.select("div.infobox-center .infobox-cell-2")
    if len(name_divs) >= 2:
        name_kr = name_divs[0].get_text(strip=True)
        name_en = name_divs[1].get_text(strip=True)
        if not all(ord(c) < 128 for c in name_kr):
            info["Romanized Name"] = name_en

    # 필드 매핑
    field_map = {
        "Romanized Name": "Romanized Name",
        "Nationality": "Nationality",
        "Born": "Born",
        "Status": "Status",
        "Role": "Role", "Roles": "Role",
        "Team": "Team", "Teams": "Team",
        "Signature Heroes": "Signature Heroes",
        "Game Appearances": "Game Appearances",
        "Approx. Total Winnings": "Approx. Total Winnings"
    }

    labels = soup.select("div.infobox-cell-2.infobox-description")
    for label in labels:
        key = label.get_text(strip=True).rstrip(":")
        value_div = label.find_next_sibling("div")
        if not value_div:
            continue

        if key == "Signature Heroes":
            value = ", ".join(tag['title'] for tag in value_div.select('a[title]'))
        else:
            value = value_div.get_text(" ", strip=True)

        if key in field_map:
            target_field = field_map[key]
            if target_field == "Romanized Name" and info[target_field]:
                continue
            info[target_field] = value

    return info


for player in all_players:
    try:
        data = get_player_info(player)
        print(data)
    except Exception as e:
        print(f"Error fetching data for {player}: {e}")


{'Player': 'tunder', 'Romanized Name': '', 'Nationality': 'Brazil', 'Born': 'April 26, 2004 (age\xa021)', 'Status': 'Active', 'Role': 'DPS', 'Team': 'ZoKorp Esports', 'Signature Heroes': 'Sojourn, Tracer, Cassidy', 'Game Appearances': 'Overwatch 2', 'Approx. Total Winnings': '$140'}
{'Player': 'speed75', 'Romanized Name': '', 'Nationality': 'Indonesia Saudi Arabia', 'Born': '', 'Status': 'Active', 'Role': 'DPS', 'Team': 'Cold Metal', 'Signature Heroes': 'Tracer, Ashe, Pharah', 'Game Appearances': 'Overwatch 2', 'Approx. Total Winnings': '$3,729'}
{'Player': 'Hesty', 'Romanized Name': 'Yoshikazu Honda', 'Nationality': 'Japan', 'Born': 'September 17, 1999 (age\xa025)', 'Status': 'Active', 'Role': 'Support', 'Team': 'REVATI', 'Signature Heroes': 'Lúcio, Baptiste, Brigitte', 'Game Appearances': 'Overwatch 2', 'Approx. Total Winnings': '$1,421'}
{'Player': 'Zeb', 'Romanized Name': '', 'Nationality': 'United States', 'Born': 'January 27, 2002 (age\xa023)', 'Status': 'Active', 'Role': 'Tank',

In [25]:
import pandas as pd

# 수집한 데이터 리스트
all_data = []
for player in all_players:
    try:
        data = get_player_info(player)
        all_data.append(data)
    except Exception as e:
        print(f"Error fetching data for {player}: {e}")

# field_map 기준으로 열 순서 지정
columns = [
    "Player", "Romanized Name", "Nationality", "Born", "Status",
    "Role", "Team", "Signature Heroes", "Game Appearances", "Approx. Total Winnings"
]

# DataFrame 생성 및 CSV 저장
df = pd.DataFrame(all_data, columns=columns)
df.to_csv("esports_players_2025.csv", index=False, encoding="utf-8-sig")